# Movie class demo
The demo walks you through the key features of the ccfepyutils movie class.

Before running this make sure you have ccfepyutils installed/in your python path.  
You will also need approprate movie settings files in your local directory "~/.ccfepytools/settings/". (Template versions of these files should be coppied there automatically when you first import ccfepyutils.)

## Getting started
First we import the class.

In [1]:
from ccfepyutils.classes.movie import Movie

# Supress logging messages
import ccfepyutils
import logging
ccfepyutils.classes.movie.logger.setLevel(logging.CRITICAL)
ccfepyutils.classes.settings.logger.setLevel(logging.CRITICAL)
ccfepyutils.classes.data_stack.logger.setLevel(logging.WARNING)
ccfepyutils.classes.plot.logger.setLevel(logging.WARNING)
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

Now we can instanciate the class by setting the movie file we want to read. We can name this movie instance 'Movie_demo' so we can keep track of what movies we are working with. This will load meta data, but not frame data.  
*NOTE: There must be a `Movie_data_locations` settings file in place in order to correctly locate the movie data for this machine and camera.*

<small>(We could also specify frame ranges and enhancements etc. here, but we'll keep things simple and do it in steps)

In [2]:
pulse = 29852
machine = 'MAST'
camera = 'SA1.1'

movie = Movie(pulse, machine, camera, name='Movie_demo');
movie

KeyError: 'nframes'

## Accessing meta data
Now we can look at the structure of the mraw file

In [ ]:
movie._movie_meta['mraw_files']

If you look at the frame meta data you'll notice the frame range has been set acording to previously set mvoie_range settings but the frame data has not been loaded - the 'set' column below is all False

In [ ]:
movie._meta

All movie settings always have a value, so if you do not set them, they will assume default values/the values used previously

In [ ]:
movie.settings.view()

## Setting the frame range
Now lets set the frame range we want to work with we want to work with and allocate the memory for that many frames. Note this is modifying the values of the Movie_range settings file.

In [ ]:
start_frame = 13
end_frame = 25
movie.set_frames(start_frame=start_frame, end_frame=end_frame)
movie

Now the frame meta data has been updated acordingly, and again no frames are set (no frame data has been loaded).

In [ ]:
movie._meta

## Accessing frame data
Now lets get a frame object for frame 16. The `repr` tells us what movie the frame is from, the image resolution and the frame number and frame time.

In [ ]:
frame16 = movie(n=16)
frame16

Now if we access the frames data the frame data will be read from disk on the fly and we will get an xarray dataset. This is a view of the dataset containing the whole movie.

In [ ]:
frame16.data

Or if you want the underlying numpy ndarray just access the dataset's values

In [ ]:
frame16.data.values

Now we can see the data for frame16 has been 'set'

In [ ]:
movie._meta

If we want to load all the frames into memory we can call `movie.load_movie_data()` or load a subset of the frames using `movie.load_movie_data(n=my_list_of_frames)`.

## Plotting frames
Now lets plot the 11th frame in the movie (ie frame24 with i=11, n=24)

In [ ]:
frame24 = movie(i=11)
frame24.plot()

# Enhancing frames
Now lets enhance this frame by extracting the forground, then applying a gaussian but, then a sharpenning opperation and plot it

In [ ]:
movie.enhance(['extract_fg', 'reduce_noise', 'sharpen'], frames=[24], keep_raw=True)

Now we can see in the frame meta data that frame 24 has been enhanced 

In [ ]:
movie._meta

In [ ]:
frame24 = movie(i=11)
frame24.plot()

Now we can load and enhance the rest of the frames

In [ ]:
movie.enhance(['extract_fg', 'reduce_noise', 'sharpen'], frames='all', keep_raw=True)
movie._meta

## Accessing raw data
We can still access the raw data because we set `keep_raw` when we applied the enhancements. Note here we are indexing the movie with the frame time.

In [ ]:
print(movie(t=0.10015, raw=True).data.values)  
print() # Or you can get the raw data from the frame object:
print(movie(t=0.10015).raw.data.values)
movie(t=0.10015, raw=True).plot()
movie(t=0.10015).plot()

## Accessing data for the whole movie
The full 3D data set can be accessed through `.data`. As usual `movie.data.values` will give you the numpy array.

In [ ]:
movie.data

The raw (unenhanced) data can accessed with `.raw_data`

In [ ]:
movie.raw_data

## Misc
Some other useful movie attributes include (Yes _underscores aren't consistent yet!):

In [ ]:
print('Number of frames: {}\n'.format(movie.nframes))
print('Frame numbers: {}\n'.format(movie.frame_numbers))
print('Frame times: {}\n'.format(movie.frame_times))
print('Frame ramge: {}\n'.format(movie._frame_range))
print('Movie file path (for fist frame/file): {}\n'.format(movie.fn_path_0))

movie_meta = movie._movie_meta
movie_meta.pop('mraw_header')
mraw_meta = movie_meta['mraw_files']
movie_meta.pop('mraw_files')
print('Movie header meta data (reduced):\n{}'.format(movie_meta))

Hopefully you now know enough to start using the movie class! More functinality coming soon!